# TensorFlow CPU training

Create a pod file for your cluster. A pod file will provide the instructions about what the cluster should run. This pod file will download Keras and run a Keras example. This example uses the TensorFlow framework. Open vi or vim and copy and paste the following content. Save this file as tf.yaml. You can use this with either TensorFlow or TensorFlow 2. To use it with TensorFlow 2, change the Docker image to a TensorFlow 2 image.



## Clone Deep Learning Containers Repo

In [ ]:
#!git clone https://github.com/aws/deep-learning-containers.git

# Setup Environment Variables

In [ ]:
import boto3

aws_region_as_slist=!curl -s http://169.254.169.254/latest/meta-data/placement/availability-zone | sed 's/\(.*\)[a-z]/\1/'
region = aws_region_as_slist.s
print('Region: {}'.format(region))

account_id=boto3.client('sts').get_caller_identity().get('Account')
print('Account ID: {}'.format(account_id))

bucket='sagemaker-{}-{}'.format(region, account_id)
print('S3 Bucket: {}'.format(bucket))

role='arn:aws:iam::{}:role/TeamRole'.format(account_id)
print('SageMaker Role ARN: {}'.format(role))

docker_repo='dlc-demo'
print('Docker Repo Name: {}'.format(docker_repo))

# Login to ECR

In [ ]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

## Create Custom ECR Repo `dlc-demo`

In [ ]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo

# Pull the Deep Learning Container for Tensorflow 2.1 Training

Available Deep Learning Container Images:  
https://github.com/aws/deep-learning-containers/blob/master/available_images.md


In [ ]:
dlc_repo_account_id='763104351884'
print(dlc_repo_account_id)

In [ ]:
train_image='763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-training:2.1.0-cpu-py36-ubuntu18.04'.format(region)
print(train_image)

In [ ]:
dlc_repo='763104351884.dkr.ecr.{}.amazonaws.com'.format(region)
print(dlc_repo)

## Login to official DLC Repo

In [ ]:
!$(aws ecr get-login --region $region --registry-ids $dlc_repo_account_id --no-include-email)

## Pull DLC 

In [ ]:
!docker system prune -a -f

In [ ]:
!docker images

In [ ]:
!docker pull $train_image

In [ ]:
!docker images

# Extend DLC to your needs

## Build Container

In [ ]:
!pygmentize ./Dockerfile

In [ ]:
docker_repo = 'dlc-demo'
docker_tag = 'bert'

bert_image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{docker_repo}:{docker_tag}'
print(bert_image_uri)

In [ ]:
!docker images

In [ ]:
!docker build --pull --no-cache -t $docker_repo:$docker_tag -f ./Dockerfile .

In [ ]:
!docker inspect $docker_repo:$docker_tag

In [ ]:
!docker images

# Push Container To ECR

In [ ]:
!docker tag $docker_repo:$docker_tag $bert_image_uri

In [ ]:
!docker images

In [ ]:
!docker push $bert_image_uri

In [ ]:
!aws ecr list-images --repository-name $docker_repo

# Define Training Job

# Create Training Job

In [ ]:
!pygmentize bert.yaml

In [ ]:
!kubectl delete -f bert.yaml

In [ ]:
!kubectl create -f bert.yaml

In [ ]:
!kubectl get pod bert

In [ ]:
!kubectl describe pod bert

In [ ]:
!kubectl logs -f bert

In [ ]:
!kubectl get pods 

In [ ]:
!pygmentize bert.yaml

In [ ]:
!kubectl delete -f bert.yaml

In [ ]:
!kubectl get pods bert

In [ ]:
!kubectl create -f bert.yaml

In [ ]:
!kubectl get pods bert

In [ ]:
!kubectl describe pods bert

In [ ]:
!kubectl logs -f bert